# Venture Funding with Deep Learning

In [1]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

---
## Preprocess the Data

In [2]:
# Reading the applicants_data.csv file from the Resources folder into a Pandas DataFrame
data = Path("./Resources/applicants_data.csv")
applicant_data_df = pd.read_csv(data)
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Reviewing the data types associated with the columns
applicant_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [4]:
# Dropping the 'EIN' and 'NAME' columns from the dataframe
applicant_data_df = applicant_data_df.drop(['EIN','NAME'],axis=1)
applicant_data_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Creating a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes == "object"].index)
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
# Creating a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [7]:
# Encoding the categorical variables
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])

In [8]:
# Creating a new dataframe with encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

# Display sample data
encoded_df.head()

C:\Users\racha\Anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Creating a DataFrame with the columnns containing numerical variables from the original dataset
numerical_variables_df = applicant_data_df.drop(columns = categorical_variables)

In [11]:
# Adding the numerical variables from the original datafram to the encoded dataframe
encoded_df = pd.concat([encoded_df, numerical_variables_df], axis=1)
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT,IS_SUCCESSFUL
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590,1


In [12]:
# Defining the target set y using the IS_SUCCESSFUL column from the original dataframe
y = applicant_data_df['IS_SUCCESSFUL']
y.head()

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [13]:
# Defining features as the encoded dataframe
X = encoded_df.drop(columns=['IS_SUCCESSFUL'])
X.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,108590
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,6692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,142590


In [14]:
# Splitting the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [15]:
# Creating a StandardScaler instance
X_scaler = StandardScaler()
# Fitting the scaler to the features training dataset
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

In [16]:
# Number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Defining the number of hidden nodes for the first hidden layer
# Using the mean of the number of input features plus the number of output neurons
hidden_nodes_layer1 =  (number_input_features + 1) // 2 

# Defining the number of hidden nodes for the second hidden layer
# Using the mean of the number of hidden nodes in the first hidden layer plus the number of output neurons
hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

# Creating the Sequential model instance
nn = Sequential()

# Adding the first hidden layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Adding the second hidden layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Adding the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=1, activation="sigmoid"))

In [17]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compiling the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Fitting the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5771 - accuracy: 0.7173
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7292
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5494 - accuracy: 0.7323
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7329
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5464 - accuracy: 0.7332
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5449 - accuracy: 0.7346
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5443 - accuracy: 0.7337
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5437 - accuracy: 0.7344
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5423 - accuracy: 0.7357
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5420 - accuracy: 0.7361

In [20]:
# Evaluating the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Displaying the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5547 - accuracy: 0.7299 - 464ms/epoch - 2ms/step
Loss: 0.5546640753746033, Accuracy: 0.729912519454956


In [21]:
# Setting the model's file path
file_path = Path("../Resources/AlphabetSoup.h5")

# Exporting the model to a HDF5 file
nn.save(file_path)

---

## Optimize the neural network model


### Alternative Model 1
#### Dropping 2 features

In [22]:
#Review features
X.describe()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,STATUS,ASK_AMT
count,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,...,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,34299.000000,3.429900e+04
mean,0.015394,0.000787,0.001924,0.000087,0.000058,0.000029,0.031050,0.000466,0.000087,0.000058,...,0.098370,0.006997,0.027843,0.109245,0.004053,0.005394,0.999213,0.000787,0.999854,2.769199e+06
std,0.123116,0.028046,0.043825,0.009352,0.007636,0.005400,0.173457,0.021594,0.009352,0.007636,...,0.297819,0.083358,0.164526,0.311951,0.063532,0.073245,0.028046,0.028046,0.012073,8.713045e+07
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000e+03
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,5.000000e+03
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,5.000000e+03
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,7.742000e+03
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.597806e+09


In [23]:
#Dropping Status and Special Considerations (No)
X = X.drop(columns=['SPECIAL_CONSIDERATIONS_N','STATUS'])
X.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y,ASK_AMT
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108590
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6692
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,142590


In [24]:
# Setting a variable to the number of features
number_input_features = len(X_train.iloc[0])
number_input_features

116

In [25]:
# Defining the number of hidden notes for layer 1 & 2
hidden_nodes_layer1_A1 = (number_input_features + 1) // 2 
hidden_nodes_layer2_A1 = (hidden_nodes_layer1 + 1) // 2

In [26]:
# Creating the Sequential model instance
nn_A1 = Sequential()

# Adding the first hidden layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn_A1.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Adding the second hidden layer specifying the number of hidden nodes and the activation function
nn_A1.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Adding the output layer to the model specifying the number of output neurons and activation function
nn_A1.add(Dense(units=1, activation="sigmoid"))

In [27]:
# Compiling the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [28]:
# Fitting the model using 50 epochs and the training data
fit_model_A1 = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5317 - accuracy: 0.7417
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5320 - accuracy: 0.7410
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5314 - accuracy: 0.7413
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5313 - accuracy: 0.7416
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5314 - accuracy: 0.7420
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5315 - accuracy: 0.7411
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5315 - accuracy: 0.7421
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5310 - accuracy: 0.7422
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5312 - accuracy: 0.7421
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5312 - accuracy: 0.7420

### Alternative Model 2
#### Using different activation functions

In [29]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [30]:
# Defining the number of hidden notes for layer 1 & 2
hidden_nodes_layer1_A2 = (number_input_features + 1) // 2 
hidden_nodes_layer2_A2 = (hidden_nodes_layer1 + 1) // 2

In [32]:
# Creating the Sequential model instance
nn_A2 = Sequential()

# Adding the first hidden layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn_A2.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="linear"))

# Adding the second hidden layer specifying the number of hidden nodes and the activation function
nn_A2.add(Dense(units=hidden_nodes_layer2, activation="linear"))

# Adding the output layer to the model specifying the number of output neurons and activation function
nn_A2.add(Dense(units=1, activation="sigmoid"))

In [33]:
# Compiling the Sequential model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Fitting the model using 50 epochs and the training data
fit_model_A2 = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5287 - accuracy: 0.7431
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5281 - accuracy: 0.7427
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5284 - accuracy: 0.7422
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5283 - accuracy: 0.7429
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5279 - accuracy: 0.7432
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5285 - accuracy: 0.7434
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5277 - accuracy: 0.7425
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5279 - accuracy: 0.7429
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5275 - accuracy: 0.7430
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5281 - accuracy: 0.7431

### Accuracy Scores by Model

In [35]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.5669 - accuracy: 0.7296 - 373ms/epoch - 1ms/step
Loss: 0.5669330358505249, Accuracy: 0.7295626997947693


In [36]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 1s - loss: 0.8140 - accuracy: 0.5415 - 588ms/epoch - 2ms/step
Loss: 0.813994288444519, Accuracy: 0.5414577126502991


In [37]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled,y_test,verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 1s - loss: 1.2385 - accuracy: 0.3913 - 525ms/epoch - 2ms/step
Loss: 1.2384518384933472, Accuracy: 0.39125365018844604


### Saving HDF5 files

In [38]:
# Setting the model's file path
file_path = Path("../Resources/AlphabetSoup_A1.h5")

# Exporting the model to a HDF5 file
nn_A1.save(file_path)

In [39]:
# Setting the model's file path
file_path = Path("../Resources/AlphabetSoup_A2.h5")

# Exporting the model to a HDF5 file
nn_A2.save(file_path)